In [56]:
import numpy as np
import ipyvolume as ipv
from compas.geometry import Point, Frame, Box, Sphere
from compas_vol.primitives import VolBox, VolSphere
from compas_vol.combinations import Union

In [57]:
Box?

In [58]:
box = Box(Frame(Point(2, 3, 1), (1, .2, -.3), (0, 1, .4)), 25, 20, 15)
vbox = VolBox(box, 4.)

In [59]:
vbox.get_distance(Point(7, 6, 9))

0.11752446224117374

In [60]:
x, y, z = np.ogrid[-20:20:128j, -20:20:128j, -20:20:128j]

In [61]:
x.shape

(128, 1, 1)

In [62]:
dm = vbox.get_distance_numpy(x, y, z)

In [63]:
dm.shape

(128, 128, 128)

In [64]:
fig = ipv.figure()
ipv.volshow(dm)
ipv.show()

In [65]:
fig = ipv.figure()
mesh = ipv.plot_isosurface(dm, 0.0, color='white')
ipv.show()

In [66]:
sphere = Sphere(Point(-4, -4, -4), 15)
vsphere = VolSphere(sphere)

In [67]:
union = Union(vbox, vsphere)

In [68]:
dm2 = union.get_distance_numpy(x, y, z)

In [69]:
fig = ipv.figure()
mesh = ipv.plot_isosurface(dm2, 0.0, color='white', extent=[(-20, 20) for _ in range(3)])
ipv.style()
ipv.show()

In [70]:
from compas_vol.microstructures import Lattice
from compas_vol.combinations import Intersection

In [71]:
help(Lattice)

Help on class Lattice in module compas_vol.microstructures.lattice:

class Lattice(builtins.object)
 |  A lattice is defined by it's type, size of a unit cell and its strut diameter.
 |  Optionally, a frame can be specified to modify orientation and origin.
 |  
 |  Parameters
 |  ----------
 |  ltype : int
 |      The index to the type of lattice.
 |      A dictionary with indices and corresponding type names can be retrieved with the property `typenames`.
 |  unitcell : float
 |      The edgelength of a cubic unit cell.
 |  thickness : float
 |      The diameter of the struts.
 |  
 |  Examples
 |  --------
 |  >>> from compas_vol.microstructure import Lattice
 |  >>> lat = Lattice(5, 5.0, 0.3)
 |  >>> lat.frame = Frame((1, 0, 0), (1, 0.2, 0.1), (-0.3, 1, 0.2))
 |  
 |  Methods defined here:
 |  
 |  __init__(self, ltype=0, unitcell=1.0, thickness=0.1)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  create_points(self)
 |      Creates a list of 20 poin

In [72]:
lattice = Lattice(1, 7., 2.)

In [73]:
lattice.typenames

{0: 'bigx',
 1: 'grid',
 2: 'star',
 3: 'cross',
 4: 'octagon',
 5: 'octet',
 6: 'vintile',
 7: 'dual',
 8: 'interlock',
 9: 'isotrop',
 10: 'hexgrid'}

In [74]:
lattice.type = 5
lattice.unicell = 6
lattice.thickness = 1
intersection = Intersection(union, lattice)

In [75]:
dm3 = intersection.get_distance_numpy(x, y, z)

In [76]:
fig = ipv.figure()
mesh = ipv.plot_isosurface(dm3, 0.0, color='white', extent=[(-20, 20) for _ in range(3)])
ipv.style()
ipv.show()

In [77]:
from compas_vol.modifications import Shell

In [78]:
shell = Shell(union, 1.5, 0.5)
union_complete = Union(intersection, shell)

In [79]:
dm4 = union_complete.get_distance_numpy(x, y, z)

In [80]:
fig = ipv.figure()
mesh = ipv.plot_isosurface(dm4, 0.0, color='white', extent=[(-20, 20) for _ in range(3)])
ipv.style()
ipv.show()

In [81]:
from compas_vol.utilities import export_ipv_mesh

In [82]:
export_ipv_mesh(mesh, './union_complete.obj')

In [83]:
from compas_vol.primitives import VolPlane, VolCylinder
from compas_vol.microstructures import TPMS
from compas.geometry import Plane, Cylinder, Circle
from compas_vol.combinations import Subtraction
from compas_vol.modifications import Overlay

In [101]:
handlebox = VolBox(Box(Frame.worldXY(), 250, 30, 10), 1.5)
endcap = VolCylinder(Cylinder(Circle(Plane((125,0,0), (0,0,1)), 15), 10))
handle = Union(handlebox, endcap)

In [102]:
rx, ry, rz = np.ogrid[-130:145:550j, -16:16:64j, -8:8:32j]

In [98]:
dmhandle = handle.get_distance_numpy(rx, ry, rz)

In [104]:
ipv.figure(width=800, height=450)
mesh = ipv.plot_isosurface(dmhandle, 0.0, extent=[[-130, 145], [-16, 16], [-8, 8]], color='pink')
ipv.xyzlim(145)
ipv.style.use('minimal')
ipv.show()

In [108]:
gyroid = TPMS(tpmstype='Gyroid', wavelength=5.0)
shell = Shell(gyroid, 2.0, 0.5)
vplane = VolPlane(Plane((0, 0, 0), (1, 0, 0)))
thickening = Overlay(shell, vplane, .005)
taper = Overlay(handle, vplane, -.01)
intersection = Intersection(thickening, taper)
outercylinder = VolCylinder(Cylinder(Circle(Plane((125,0,0), (0,0,1)), 12), 13))
innercylinder = VolCylinder(Cylinder(Circle(Plane((125,0,0), (0,0,1)), 10), 20))
add = Union(intersection, outercylinder)
finalresult = Subtraction(add, innercylinder)

In [109]:
wrenchdm = finalresult.get_distance_numpy(rx, ry, rz)

In [112]:
ipv.figure(width=800, height=450)
mesh = ipv.plot_isosurface(wrenchdm, 0.0, extent=[[-130, 145], [-16, 16], [-8, 8]], color='pink')
ipv.xyzlim(145)
ipv.style.use('minimal')
ipv.show()